In [1]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F

class LinearRegressionClient:
    
    class LinearRegression(torch.nn.Module):
    
        def __init__(self, in_layer, out_layer, criterion, optimizer):
            super(LinearRegression, self).__init__()
            self.linear = torch.nn.Linear(in_layer, out_layer, bias=False)
            self.criterion = criterion
            self.optimizer = optimizer
        
        def forward(self, x):
            y_pred = self.linear(x)
            return y_pred

        def train_model(self, epochs, x_data, y_data):
            for epoch in range(epochs):
                #Set model ready to train
                self.linear.train()
                self.optimizer.zero_grad()
                # Forward pass
                y_pred = self.linear(x_data)
                # Compute Loss
                loss = self.criterion(y_pred, y_data)
                # Backward pass
                loss.backward()
                self.optimizer.step()
            print("Loss: "+ str(loss.data.view(-1)))

    
        def set_parameters_from_tensor(self, weights: torch.Tensor):
            if not isinstance(weights, torch.Tensor):
                print("Converting weights to tensor type")
                weights = torch.tensor(weights)
            current_index = 0 
            for parameter in self.parameters():
                numel = parameter.data.numel()
                size = parameter.data.size()
                parameter.data.copy_(
                                weights[current_index:current_index +
                                                    numel].view(size))
                current_index+=1

        def get_parameters_as_tensor(self):
            params = [param.data.view(-1) for param in self.parameters()]
            params = torch.cat(params)
            params = params.cpu()
            return params

    def run(in_layer, out_layer, epochs, data, learn_rate=0.1, criterion=None, optimizer=None, set_parameters=False, incoming_parameters=None):
        criterion = torch.nn.MSELoss(size_average=False)
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=learn_rate)

        model = LinearRegression(in_layer, out_layer, criterion, optimizer)

        if set_parameters:
            model.set_parameters_as_tensor(incoming_parameters)
        
        model.train_model(epochs, data[0], data[1])

        return model.get_parameters_as_tensor()





In [5]:
class LinearRegressionFederated:

    def get_average_model(self, model_parameters):
        
        #TODO: Push list of model parameters to some SCN
        for param in model_parameters:
            break
            # param.send(models[random])

        #TODO: Calculate the Mean Remotely using existing safe object
        model_sum = 0.
        for param in model_parameters:
            model_sum += param
        model_param_avg = model_sum/len(model_parameters)

        #TODO: Pull result to orchestrator
        return model_param_avg
    
    def train_cycle(self, in_layer, out_layer, epochs, first_cycle=True, avg_parameters=None):
        #train each model fresh
        if first_cycle:
            avg_parameters = []
            for i in range(len(self.data_federation)):
                models.append(LinearRegressionClient.run(in_layer, out_layer, epochs, self.data_federation[i], None, None, False, None))
        #train each model based on previous average
        else:
            for i in range(len(self.data_federation)):
                models.append(LinearRegressionClient.run(in_layer, out_layer, epochs, self.data_federation[i], None, None, True, avg_model))


        #average the models and return
        return self.get_average_model(models)

    def run(data_federation, in_layer, out_layer, epochs, batches,):
                
        self.data_federation = data_federation
        avg_model = train_cycle(in_layer, out_layer, epochs)
        for batch in range(batches-1):
            train_cycle(in_layer, out_layer, epochs, True, avg_model)
        



In [6]:
#Data Starts with respective owners A and B
x_dataA = Variable(torch.Tensor([[10.0], [9.0], [3.0], [2.0]]))
y_dataA = Variable(torch.Tensor([[90.0], [80.0], [50.0], [30.0]]))
x_dataB = Variable(torch.Tensor([[7.0], [8.0], [3.0], [1.0]]))
y_dataB = Variable(torch.Tensor([[60.0], [70.0], [40.0], [20.0]]))
data_federation = [[x_dataA, y_dataA], [x_dataB, y_dataB]]
        
# modelA = LinearRegressionClient(1,1)
# modelB = LinearRegressionClient(1,1)
# models = [modelA, modelB]

federated_model = LinearRegressionFederated(models, data_federation)

TypeError: __init__() missing 2 required positional arguments: 'criterion' and 'optimizer'

In [312]:
for i in range(10):
    # print(federated_model.models[0].get_parameters_as_tensor())
    # print(federated_model.models[1].get_parameters_as_tensor())
    federated_model.train_cycle(1)
    # print(federated_model.models[0].get_parameters_as_tensor())
    # print(federated_model.models[1].get_parameters_as_tensor())

Loss: tensor([1.3727e+34])
Loss: tensor([8.7761e+33])
Loss: tensor([1.3438e+37])
Loss: tensor([8.5908e+36])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
Loss: tensor([inf])
